<h2>Objective</h2>

<h3> the main objectuve of this notebook is to make use tranfer learning to make a working protoype model of image classification on the small sample images of plants.</h3>

**evaluation metric**

<u>if the we can make a small dataset from the whole dataset and train any model on basis of it .</u>

then we can use those functions to train various models and find out the best model for our dataset.

In [1]:
# ignoring all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing the necessary libraries

import numpy as np
import pandas as pd
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2023-04-16 14:07:06.251163: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 14:07:08.165885: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 14:07:08.168900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 14:07:12.789625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# importing the metadata.csv with all the path to images and their labels

tdata=pd.read_csv(r'/home/drowsy/Downloads/data/PlantCLEF2022_trusted_training_metadata.csv',delimiter=';')
wdata=pd.read_csv(r'/home/drowsy/Downloads/data/PlantCLEF2022_web_training_metadata.csv',delimiter=';')

In [4]:
# directory of the images to train the model.

data_dir='/home/drowsy/Downloads/data/train/images'

# converting the path to pathlib for easy anvigation

import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/home/drowsy/Downloads/data/train/images')

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [5]:
tdata.head(1)

,classid,image_name,image_path,species,genus,family,order,class,source,manual_tag,predicted_tag,predicted_tag_probability,original_url,license,publisher,gbif_occurrence_id,aggregator,dataset_key,image_backup_url
0,2683260,d0749fe4f8ade13dd9402b0f43bc29e8f28af27a.jpg,2683260/d0749fe4f8ade13dd9402b0f43bc29e8f28af2...,Cycas angulata R.Br.,Cycas,Cycadaceae,Cycadales,Cycadopsida,NaN,NaN,habit,0.7386,https://images.ala.org.au/image/proxyImageThum...,http://creativecommons.org/licenses/by/4.0/,ALA species sightings and OzAtlas,1.632965e+09,gbif,84a649ce-ff81-420d-9c41-aa1de59e3766,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...


In [5]:
# concating the classid and species of the images
tdata['labels']=tdata['classid'].astype('str')+'$'+tdata['species'].astype('str')

In [7]:
len(tdata['classid'].unique())

80000

In [8]:
len(tdata['labels'].unique())

80000

In [6]:
labels_df2=tdata['labels'].unique()
type(labels_df2)

numpy.ndarray

In [7]:
# making a dict with classid as key and name of plant species as value

def get_labels(labels):
    label_dict={}
    for i in range(len(labels)):
        label_dict[str(labels[i]).split('$')[0]]=str(labels[i]).split('$')[1]
    return label_dict

plant_all_label_dict=get_labels(labels_df2)
plant_all_label_dict

{'2683260': 'Cycas angulata R.Br.',
 '2683214': 'Cycas armstrongii Miq.',
 '2683283': 'Cycas beddomei Dyer',
 '2683262': 'Cycas calcicola Maconochie',
 '2683209': 'Cycas candida K.D.Hill',
 '2683264': 'Cycas circinalis L.',
 '2683259': 'Cycas clivicola K.D.Hill',
 '2683265': 'Cycas debaoensis Y.C.Zhong & C.J.Chen',
 '2683238': 'Cycas edentata de Laub.',
 '2683293': 'Cycas falcata K.D.Hill',
 '2683245': 'Cycas indica A.Lindstr. & K.D.Hill',
 '2683234': 'Cycas inermis Lour.',
 '2683252': 'Cycas media R.Br.',
 '2683207': 'Cycas micronesica K.D.Hill',
 '2683250': 'Cycas pectinata Buch.-Ham.',
 '2683254': 'Cycas pruinosa Maconochie',
 '2683217': 'Cycas revoluta Thunb.',
 '2683272': 'Cycas rumphii Miq.',
 '2683297': 'Cycas seemannii Braun',
 '2683286': 'Cycas siamensis Miq.',
 '2683228': 'Cycas tropophylla K.D.Hill & P.K.Lôc',
 '2683266': 'Cycas zeylanica A.Lindstr. & K.D.Hill (J.Schust.)',
 '5284011': 'Bowenia serrulata Chamb. (W.Bull)',
 '5284014': 'Bowenia spectabilis Hook. ex Hook.f.',
 

In [11]:
len(plant_all_label_dict)

80000

In [ ]:
# giving list of dir path of 2000 images at a time

In [52]:
l1=list(data_dir.glob('*'))[:4]

In [55]:
list(l1[1].glob('*'))

[PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b8303bde93d6695633c9d3d2e22d9be7ff72f294.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/066e3deb088f1e237c888ea4ae83b49e0133bb26.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b899c226208eb26cc1df86ef26f4b8190cc80a4b.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/e5183747cf5f6766ef5961e2ddd29e55dbaa5da9.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b453c53128f56df39a4d2f8396f060fc6a3f200e.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8c49e879b74cdbf216554da3acd4bccb4b3fbf1d.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8a611b275588ba9507cb7fea9890b4ced4712eca.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/3e2e18a68ecc543968e6d94683893a4472089944.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/23f8e78db7214be43be3ea89af16bf3dd02f3f89.jpg'),
 PosixPath

In [ ]:
# made a function that will generate dict for given number of path in a list

In [8]:
# making a dict with name of folder as key 
# and putting the path of all images inside the folder as a list in the dict as values.

def get_images1(labels):
    images_dict={}
#     labels are the list of all the path of subfolders in directory
#     labels=list(dirc.glob('*'))
    for i in range(len(labels)):
#         putting the name of folder inside main dir in key of dict 
#         putting all the images in the folder in list , in value of dict
        images_dict[str(labels[i]).split('/')[-2]]=list(labels[i].parent.glob('*'))
    return images_dict

get_images1(list(data_dir.glob('*/*'))[:20])

{'10763249': [PosixPath('/home/drowsy/Downloads/data/train/images/10763249/9658cb7a16923306429d2b918b32739dff32067d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10763249/c47a91dd800fcee231b9e4671f4f82cf0a11d5fd.jpg')],
 '10696491': [PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b8303bde93d6695633c9d3d2e22d9be7ff72f294.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/066e3deb088f1e237c888ea4ae83b49e0133bb26.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b899c226208eb26cc1df86ef26f4b8190cc80a4b.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/e5183747cf5f6766ef5961e2ddd29e55dbaa5da9.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b453c53128f56df39a4d2f8396f060fc6a3f200e.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8c49e879b74cdbf216554da3acd4bccb4b3fbf1d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8a611b275588ba9507cb7fea

In [57]:
list(path_list[9].parent.glob('*'))

[PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b8303bde93d6695633c9d3d2e22d9be7ff72f294.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/066e3deb088f1e237c888ea4ae83b49e0133bb26.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b899c226208eb26cc1df86ef26f4b8190cc80a4b.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/e5183747cf5f6766ef5961e2ddd29e55dbaa5da9.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b453c53128f56df39a4d2f8396f060fc6a3f200e.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8c49e879b74cdbf216554da3acd4bccb4b3fbf1d.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8a611b275588ba9507cb7fea9890b4ced4712eca.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/3e2e18a68ecc543968e6d94683893a4472089944.jpg'),
 PosixPath('/home/drowsy/Downloads/data/train/images/10696491/23f8e78db7214be43be3ea89af16bf3dd02f3f89.jpg'),
 PosixPath

In [9]:
path_list=list(data_dir.glob('*/*'))[:200]
plant_2kimages_dict=get_images1(path_list)

In [22]:
plant_2kimages_dict

{'10763249': [PosixPath('/home/drowsy/Downloads/data/train/images/10763249/9658cb7a16923306429d2b918b32739dff32067d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10763249/c47a91dd800fcee231b9e4671f4f82cf0a11d5fd.jpg')],
 '10696491': [PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b8303bde93d6695633c9d3d2e22d9be7ff72f294.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/066e3deb088f1e237c888ea4ae83b49e0133bb26.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b899c226208eb26cc1df86ef26f4b8190cc80a4b.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/e5183747cf5f6766ef5961e2ddd29e55dbaa5da9.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b453c53128f56df39a4d2f8396f060fc6a3f200e.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8c49e879b74cdbf216554da3acd4bccb4b3fbf1d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8a611b275588ba9507cb7fea

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [17]:
# getting all the classid of images present in the folder

labels_path=list(data_dir.glob('*'))
labels=[]
for i in range(len(labels_path)):
    labels.append(str(labels_path[i]).split('/')[-1])
labels_df = pd.DataFrame(labels,columns=['classid'])
labels_df.head()

,classid
0,10763249
1,10696491
2,11208838
3,10780269
4,10953587


In [18]:
len(labels_df)

712

In [10]:
# changing the column classid to number to merge with the metadata

labels_df['classid']=pd.to_numeric(labels_df['classid'],downcast='integer')

In [19]:
# getting only 2 columns specifically from the dataframe

tdata[['species','classid']].head(1)

,species,classid
0,Cycas angulata R.Br.,2683260


In [12]:
# merging the labels from the classid's present in the folder with in metadata 
# and getting species based on it

merged_labels=labels_df.merge(tdata[['species','classid']],on=['classid'])

In [13]:
merged_labels.head(1)

,classid,species
0,10763249,Miconia allenii Almeda (Almeda)


In [14]:
# concatenating the classid with species to get unique values 
# i.e removing the duplicate values and getting the labels/unique classes of plants in the data folder

merged_labels['labels']=merged_labels['classid'].astype('str')+'$'+merged_labels['species'].astype('str')

In [15]:
# droping the columns that are not needed

merged_labels.drop(columns=['classid','species'],inplace=True)

In [16]:
# m_labels=np.array(m_labels)
m_labels=pd.DataFrame(m_labels)
type(m_labels)

pandas.core.frame.DataFrame

In [19]:
m_labels

pandas.core.frame.DataFrame

In [17]:
# getting the unique values/ classes

m_labels=m_labels[0].unique()

KeyError: 0

In [ ]:
# total number of classes

len(m_labels)

In [ ]:
m_labels

In [ ]:
str(m_labels[0]).split('$')[1]

--------------------------------------------------------------------------------------------------------------------

In [37]:
# making a dict with name of folder as key 
# and putting the path of all images inside the folder as a list in the dict as values.

def get_images(dirc):
    images_dict={}
#     labels are the list of all the path of subfolders in directory
    labels=list(dirc.glob('*'))
    for i in range(len(labels)):
#         putting the name of folder inside main dir in key of dict 
#         putting all the images in the folder in list , in value of dict
        images_dict[str(labels[i]).split('/')[-1]]=list(labels[i].glob('*'))
    return images_dict

plant_images_dict = get_images(data_dir)
plant_images_dict

{'10763249': [PosixPath('/home/drowsy/Downloads/data/train/images/10763249/9658cb7a16923306429d2b918b32739dff32067d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10763249/c47a91dd800fcee231b9e4671f4f82cf0a11d5fd.jpg')],
 '10696491': [PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b8303bde93d6695633c9d3d2e22d9be7ff72f294.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/066e3deb088f1e237c888ea4ae83b49e0133bb26.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b899c226208eb26cc1df86ef26f4b8190cc80a4b.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/e5183747cf5f6766ef5961e2ddd29e55dbaa5da9.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b453c53128f56df39a4d2f8396f060fc6a3f200e.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8c49e879b74cdbf216554da3acd4bccb4b3fbf1d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8a611b275588ba9507cb7fea

In [ ]:
plant_images_dict

In [ ]:
# making a dict with classid as key and name of plant species as value

def get_labels(labels):
    label_dict={}
    for i in range(len(labels)):
        label_dict[str(labels[i]).split('$')[0]]=str(labels[i]).split('$')[1]
    return label_dict

plant_label_dict=get_labels()

In [ ]:
plant_label_dict

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [10]:
# processing the images into a list for training set 
# which will be used for training the model

def img_read(image_dict,label_dict):
    x=[]
    y=[]
    sum1=0
    for nameid,images in image_dict.items():
        for image in images:
            img=cv2.imread(str(image))
            resized_img=cv2.resize(img,(512,512))
            x.append(resized_img)
            y.append(label_dict[nameid])
            sum1=sum1+1
            print('number : ',sum1)
        
    x=np.array(x)
    y=np.array(y)
    return x,y

In [15]:
plant_all_label_dict

{'2683260': 'Cycas angulata R.Br.',
 '2683214': 'Cycas armstrongii Miq.',
 '2683283': 'Cycas beddomei Dyer',
 '2683262': 'Cycas calcicola Maconochie',
 '2683209': 'Cycas candida K.D.Hill',
 '2683264': 'Cycas circinalis L.',
 '2683259': 'Cycas clivicola K.D.Hill',
 '2683265': 'Cycas debaoensis Y.C.Zhong & C.J.Chen',
 '2683238': 'Cycas edentata de Laub.',
 '2683293': 'Cycas falcata K.D.Hill',
 '2683245': 'Cycas indica A.Lindstr. & K.D.Hill',
 '2683234': 'Cycas inermis Lour.',
 '2683252': 'Cycas media R.Br.',
 '2683207': 'Cycas micronesica K.D.Hill',
 '2683250': 'Cycas pectinata Buch.-Ham.',
 '2683254': 'Cycas pruinosa Maconochie',
 '2683217': 'Cycas revoluta Thunb.',
 '2683272': 'Cycas rumphii Miq.',
 '2683297': 'Cycas seemannii Braun',
 '2683286': 'Cycas siamensis Miq.',
 '2683228': 'Cycas tropophylla K.D.Hill & P.K.Lôc',
 '2683266': 'Cycas zeylanica A.Lindstr. & K.D.Hill (J.Schust.)',
 '5284011': 'Bowenia serrulata Chamb. (W.Bull)',
 '5284014': 'Bowenia spectabilis Hook. ex Hook.f.',
 

In [69]:
len(plant_all_label_dict)

80000

In [16]:
plant_2kimages_dict

{'10763249': [PosixPath('/home/drowsy/Downloads/data/train/images/10763249/9658cb7a16923306429d2b918b32739dff32067d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10763249/c47a91dd800fcee231b9e4671f4f82cf0a11d5fd.jpg')],
 '10696491': [PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b8303bde93d6695633c9d3d2e22d9be7ff72f294.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/066e3deb088f1e237c888ea4ae83b49e0133bb26.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b899c226208eb26cc1df86ef26f4b8190cc80a4b.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/e5183747cf5f6766ef5961e2ddd29e55dbaa5da9.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/b453c53128f56df39a4d2f8396f060fc6a3f200e.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8c49e879b74cdbf216554da3acd4bccb4b3fbf1d.jpg'),
  PosixPath('/home/drowsy/Downloads/data/train/images/10696491/8a611b275588ba9507cb7fea

In [24]:
count=0
for i,j in plant_2kimages_dict.items():
    for k in j:
        count+=1
    
count

296

In [11]:
X,y=img_read(plant_2kimages_dict,plant_all_label_dict)

number :  1
number :  2
number :  3
number :  4
number :  5
number :  6
number :  7
number :  8
number :  9
number :  10
number :  11
number :  12
number :  13
number :  14
number :  15
number :  16
number :  17
number :  18
number :  19
number :  20
number :  21
number :  22
number :  23
number :  24
number :  25
number :  26
number :  27
number :  28
number :  29
number :  30
number :  31
number :  32
number :  33
number :  34
number :  35
number :  36
number :  37
number :  38
number :  39
number :  40
number :  41
number :  42
number :  43
number :  44
number :  45
number :  46
number :  47
number :  48
number :  49
number :  50
number :  51
number :  52
number :  53
number :  54
number :  55
number :  56
number :  57
number :  58
number :  59
number :  60
number :  61
number :  62
number :  63
number :  64
number :  65
number :  66
number :  67
number :  68
number :  69
number :  70
number :  71
number :  72
number :  73
number :  74
number :  75
number :  76
number :  77
number :

In [ ]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X=sc.fit_transform(X)
x

In [ ]:
X=X/255.0

In [ ]:
# from sklearn.model_selection import train_test_split

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# `pretrained model with input image shape 512x512 , without final layer`

In [15]:
pre_trained_model=hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
                                 input_shape=(512, 512, 3), trainable=True)


In [7]:
num_of_plants = len(plant_all_labels_dict)

NameError: name 'plant_all_labels_dict' is not defined

In [16]:
model=keras.models.Sequential(
    pre_trained_model,
    layers.Dense(80000)
)

In [17]:
model.compile()

In [18]:
model.summary()

Model: "<keras.layers.core.dense.Dense object at 0x7f000021dbd0>"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
Total params: 207,615,832
Trainable params: 206,838,808
Non-trainable params: 777,024
_________________________________________________________________


In [19]:
model.fit(X,y)

2023-04-16 14:21:43.266133: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 232783872 exceeds 10% of free system memory.


TypeError: in user code:

    File "/home/drowsy/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/drowsy/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/drowsy/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/drowsy/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/drowsy/anaconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/drowsy/anaconda3/lib/python3.10/site-packages/keras/engine/base_layer.py", line 2798, in _name_scope
        name

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [13]:
from joblib import load,dump

# dump(model,filename='./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib')

In [14]:
loaded_model=load(filename='model_pre_name.joblib')

ValueError: Unknown layer: 'KerasLayer'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.